In [32]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [24]:
"""
Bitcoin Mining: Solve complex puzzles to validate transactions and add them to blockchain

puzzles involve computing SHA-256 Hashes over and over

1 Hash = 1 attempt to solve the puzzle

Hashrate = number of hashes/second


Antminer = 110 TH/s which is 110 trillion guesses per second

More hashrate = more revenue but more electricity usage


"""

'\nBitcoin Mining: Solve complex puzzles to calidate transactions and add them to blockchain\n\npuzzles involve computing SHA-256 Hashes over and over\n\n1 Hash = 1 attempt to solve the puzzle\n\nHashrate = number of hashes/second\n\n\nAntminer = 110 TH/s which is 110 trillion guesses per second\n\nMore hashrate = more revenue but more electricity usage\n\n\n'

In [41]:
import requests

def get_btc_price():
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {
        'ids': 'bitcoin',
        'vs_currencies': 'usd'
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data['bitcoin']['usd']

# Example usage
btc_price = get_btc_price()
print(f"Current BTC Price: ${btc_price}")

Current BTC Price: $104460


In [44]:
def load_clean_NSP_NW_da_data():
    da = pd.read_csv('/Users/kunjshah/Downloads/BTC_Mining_Project/1908b1f8-bc4f-44fa-9adf-966461c248e6.csv')
    da['interval_start_local'] = pd.to_datetime(da['interval_start_local'])
    da['interval_end_local'] = pd.to_datetime(da['interval_end_local'])
    da['interval_start_utc'] = pd.to_datetime(da['interval_start_utc'])
    da.sort_values('interval_start_local', inplace=True)
    return da

In [46]:
da= load_clean_NSP_NW_da_data()
da.to_parquet('DA_MISO_NSP_NWELOAD.parquet')

In [48]:
def load_clean_NSP_NW_rt_data():
    rt = pd.read_csv('/Users/kunjshah/Downloads/BTC_Mining_Project/773a7536-3561-464f-81b2-4bed1091d06d.csv')
    rt['interval_start_local'] = pd.to_datetime(rt['interval_start_local'])
    rt['interval_end_local'] = pd.to_datetime(rt['interval_end_local'])
    rt['interval_start_utc'] = pd.to_datetime(rt['interval_start_utc'])
    rt.sort_values('interval_start_local', inplace=True)

    return rt


In [49]:
rt = load_clean_NSP_NW_rt_data()
rt.to_parquet('RT_MISO_NSP_NWRT.parquet')

In [13]:
def load_btc_data():
    x = pd.read_csv('btc_hourly.csv')
    x['time_period_end'] = pd.to_datetime(x['time_period_end'])
    x['time_period_start'] = pd.to_datetime(x['time_period_start'])
    x['time_close'] = pd.to_datetime(x['time_close'])
    return x


In [ ]:
#Assumptions
btc_data = load_btc_data() #history of btc_data
btc_price = get_btc_price() #price btcusd current
power_consumption =3250 #watts
hashrate = 110 #TH/s
num_antminer = 1000
rf = 0.04 #annualized rf rate


rt = load_clean_NSP_NW_da_data()
da = load_clean_NSP_NW_rt_data()

btc_day = 0.00008


breakeven = 95 #$/MWH when btc 93k


In [ ]:
"""
Questions to answer:

- Estimate BTC/USD Volatility

Strategy to be profitable?
- Forecast profits for July 2025 - June 2026
- Forecast some weather
- Location of facility
- Purchase power in DA and if not used sell in RT
- Or purchase less power in DA if need more buy in RT

"""

In [ ]:
"""
Across 500 simulation what is the expected average annual excess profit (profit - rf rate)
- forecast energy prices for 1 year
- forecast btc prices for 1 year
-

What is the volatility in annual profitability
What is the sharpe


Would you buy the facility for 500k?
-Base this off of the return given the volatility of BTC versus electricity
- What is the breakeven expected value
"""

In [ ]:
"""
Intraday Analysis
- Choose an hourly intraday period as well as a shorter intraday period
- for reahc period what is the value of operating the BTC facility? What was the value of curtailment? What was the net payout?

-Model BTC and Electricity volatility surface


"""

In [ ]:
#part 2 change the node